In [1]:
import pandas as pd
from datetime import date
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam

2023-04-20 22:03:47.163749: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-20 22:03:47.196839: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-20 22:03:47.197448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 22:03:47.827295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def read_archive(path_arquivo):
    lines = [] 
    with open(path_arquivo) as arquivo:
        for line in arquivo:
            lines.append(line.split())
    df = pd.DataFrame(lines, columns= ['DIA', 'MES', 'ANO', 'MEDICAO'])
    return df

def dados(dado):
    ano = dado['ANO']
    mes = dado['MES']
    dia = dado['DIA']
    medidas = {"DATA":[], "MEDICAO":[]}
    for med in range(len(dado['ANO'])):
        if float(dado['MEDICAO'][med]) > 0:
            d = int(dia[med])
            m = int(mes[med])
            medidas['DATA'].append(date(int(ano[med]), m, d))
            medidas['MEDICAO'].append(float(dado['MEDICAO'][med]))
    return medidas

# converte um pandas dataframe no input apropriado para o LSTM
def datasetLSTM(dataset, quantidade): # df, range da data
    df = []
    param1 = 0
    param2 = 0
    for i in range(len(dataset)):
        lista=[]
        try:
            lista.append(dataset[i])
            for j in range (quantidade):
                lista.append(dataset[i+j+1])
                param1 = i
                param2 = j
            lista.append(dataset[param1+param2+15])
            param1=0
            param2=0            
            df.append(lista)
        except:
            print(f"erro no indexador {i}/{len(dataset)}")
    return df

In [3]:
%cd ..

df = read_archive("15700000.txt")
df = pd.DataFrame(dados(df))

/home/fernandoarkade/Documents/IA_rioMadeira/VisualizacaoDeDadosRioMadeira


In [4]:
df

,DATA,MEDICAO
0,2001-01-01,1803.0
1,2001-01-02,1794.0
2,2001-01-03,1799.0
3,2001-01-04,1808.0
4,2001-01-05,1818.0
...,...,...
5108,2014-12-27,1982.0
5109,2014-12-28,1996.0
5110,2014-12-29,2017.0
5111,2014-12-30,2042.0


In [5]:
# removendo a coluna "DATA" e atribuindo-a a variável date_time
date_time = pd.to_datetime(df.pop('DATA'), format='%Y-%m-%d')
# convertendo em float32
df = df.astype('float32')
# normalizando
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

# criando input para o LSTM a partir do dataframe

r = 14 # range de dias 
d= 15 # dias a partir de r + 1


df2 = datasetLSTM(df, r)

# dataset de teste
dfTest = pd.DataFrame(df2)
dfTest = dfTest.astype("float")
colunas = [i for i in range(r+1)] 
colunas.append(str(r+1+d))
dfTest.columns = colunas

dfTest

erro no indexador 5085/5113
erro no indexador 5086/5113
erro no indexador 5087/5113
erro no indexador 5088/5113
erro no indexador 5089/5113
erro no indexador 5090/5113
erro no indexador 5091/5113
erro no indexador 5092/5113
erro no indexador 5093/5113
erro no indexador 5094/5113
erro no indexador 5095/5113
erro no indexador 5096/5113
erro no indexador 5097/5113
erro no indexador 5098/5113
erro no indexador 5099/5113
erro no indexador 5100/5113
erro no indexador 5101/5113
erro no indexador 5102/5113
erro no indexador 5103/5113
erro no indexador 5104/5113
erro no indexador 5105/5113
erro no indexador 5106/5113
erro no indexador 5107/5113
erro no indexador 5108/5113
erro no indexador 5109/5113
erro no indexador 5110/5113
erro no indexador 5111/5113
erro no indexador 5112/5113


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,30
0,0.388951,0.383878,0.386697,0.391770,0.397407,0.405299,0.414882,0.421082,0.427283,0.435175,0.440812,0.449267,0.458286,0.470124,0.485907,0.636979
1,0.383878,0.386697,0.391770,0.397407,0.405299,0.414882,0.421082,0.427283,0.435175,0.440812,0.449267,0.458286,0.470124,0.485907,0.503382,0.641488
2,0.386697,0.391770,0.397407,0.405299,0.414882,0.421082,0.427283,0.435175,0.440812,0.449267,0.458286,0.470124,0.485907,0.503382,0.519729,0.646561
3,0.391770,0.397407,0.405299,0.414882,0.421082,0.427283,0.435175,0.440812,0.449267,0.458286,0.470124,0.485907,0.503382,0.519729,0.534386,0.652198
4,0.397407,0.405299,0.414882,0.421082,0.427283,0.435175,0.440812,0.449267,0.458286,0.470124,0.485907,0.503382,0.519729,0.534386,0.551860,0.658963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,0.332018,0.339910,0.347802,0.353439,0.357948,0.362458,0.370913,0.385006,0.396843,0.408681,0.420519,0.428410,0.435175,0.445321,0.458850,0.489853
5081,0.339910,0.347802,0.353439,0.357948,0.362458,0.370913,0.385006,0.396843,0.408681,0.420519,0.428410,0.435175,0.445321,0.458850,0.463923,0.497745
5082,0.347802,0.353439,0.357948,0.362458,0.370913,0.385006,0.396843,0.408681,0.420519,0.428410,0.435175,0.445321,0.458850,0.463923,0.463360,0.509583
5083,0.353439,0.357948,0.362458,0.370913,0.385006,0.396843,0.408681,0.420519,0.428410,0.435175,0.445321,0.458850,0.463923,0.463360,0.461105,0.523675


In [6]:
# train/ test split
test_size = int(len(dfTest) * 0.2)
test = dfTest.iloc[-test_size::].copy()
train = dfTest.iloc[:-test_size:].copy()


# dropando target
X_train = train.drop("30",axis=1).copy()
y_train = train[["30"]].copy()

# transformando em numpy array
scaled_X_train = X_train.to_numpy()
scaled_y_train = y_train.to_numpy()

# transformando array
scaled_y_train = scaled_y_train.reshape(-1)
scaled_y_train = np.insert(scaled_y_train, 0, 0)
scaled_y_train = np.delete(scaled_y_train, -1)

# entradas do modelo
n_input = 15
n_features= X_train.shape[1]
b_size = 32
generator = TimeseriesGenerator(scaled_X_train, scaled_y_train, length=n_input, batch_size=b_size)

# validação 

y_test = test[["30"]]
scaled_y_test = y_test.to_numpy()
X_test = test.drop("30",axis=1).copy()
scaled_X_test  = X_test.to_numpy()
test_generator = TimeseriesGenerator(scaled_X_test, scaled_y_test, length=n_input, batch_size=b_size)

print(f"generator {generator[0][0].shape}")

generator (32, 15, 15)


In [8]:
basemodel = Sequential()
basemodel.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
basemodel.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=0.001)
basemodel.compile(optimizer=optimizer, loss='mse')
basemodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 150)               99600     
                                                                 
 dense (Dense)               (None, 1)                 151       
                                                                 
Total params: 99,751
Trainable params: 99,751
Non-trainable params: 0
_________________________________________________________________


/home/fernandoarkade/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [40]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import r2_score
# baseado nos parametros do modelo base
param_grid = {'units': [50, 100, 150],
              'dropout': [0.2, 0.3, 0.4],
              'optimizer': ['adam', 'rmsprop']
             }
def create_model(units=150, dropout=0.2, optimizer= Adam(lr=0.001), n_input= 15, n_features= 15):
    model = Sequential()
    model.add(LSTM(units, activation = 'relu', input_shape=(n_input, n_features)))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# quantas vezes o modelo vai rodar até convergir
epocas = 30

#batch size, quantidade de exemplos em cada iteração
ex = generator.batch_size

# criar o modelo usando KerasRegressor
model = KerasRegressor(build_fn=create_model, epochs=epocas, batch_size=ex, verbose=0)

# cross validation
folds = 3
# realizar o grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=folds)


#X_train = np.reshape(generator.data, (0, 15, generator.data.shape[1]))
#y_train = np.reshape(generator.targets, (generator.targets.shape[0], generator.targets.shape[1], 1))
grid_result = grid.fit(generator.data, generator.targets)

# Avaliar os modelos

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/tmp/ipykernel_36426/934198518.py:24: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, epochs=epocas, batch_size=ex, verbose=0)


ValueError: 
All the 54 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/fernandoarkade/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/fernandoarkade/.local/lib/python3.10/site-packages/keras/wrappers/scikit_learn.py", line 175, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/home/fernandoarkade/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/tmp/__autograph_generated_filei9u2fht1.py", line 15, in tf__train_function
    retval_ = ag__.converted_call(ag__.ld(st

In [46]:
generator.data.shape

(4068, 15)

In [10]:
basemodel.fit_generator(generator,epochs=150)

Epoch 1/150


/tmp/ipykernel_36426/3612971751.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  basemodel.fit_generator(generator,epochs=150)
2023-04-20 22:04:13.519089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


127/127 [==============================] - 1s 5ms/step - loss: 0.0483
Epoch 2/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0182
Epoch 3/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0127
Epoch 4/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0101
Epoch 5/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0076
Epoch 6/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0056
Epoch 7/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0055
Epoch 8/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0044
Epoch 9/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0065
Epoch 10/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0051
Epoch 11/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0049
Epoch 12/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0047
Epoch 13/150
127/127 [===

127/127 [==============================] - 1s 6ms/step - loss: 0.0024
Epoch 100/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0021
Epoch 101/150
127/127 [==============================] - 1s 7ms/step - loss: 0.0025
Epoch 102/150
127/127 [==============================] - 1s 7ms/step - loss: 0.0025
Epoch 103/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0020
Epoch 104/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0022
Epoch 105/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0021
Epoch 106/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0021
Epoch 107/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0022
Epoch 108/150
127/127 [==============================] - 1s 5ms/step - loss: 0.0020
Epoch 109/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0020
Epoch 110/150
127/127 [==============================] - 1s 6ms/step - loss: 0.0020
Epoch 

In [ ]:

loss_per_epoch = basemodel.history.history['loss']
plt.figure(figsize=(16,9))
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)
!

In [ ]:
y_test = test[["30"]]
scaled_y_test = y_test.to_numpy()
scaled_y_test

In [ ]:
X_test = test.drop("30",axis=1).copy()
scaled_X_test  = X_test.to_numpy()
test_generator = TimeseriesGenerator(scaled_X_test, scaled_y_test, length=n_input, batch_size=b_size)

In [ ]:
y_pred_scaled = model.predict(test_generator)

In [ ]:
results = pd.DataFrame({'Dados Reais':test["30"].values[n_input:],'Dados Previstos':y_pred_scaled.ravel()})

In [ ]:
results.plot(title="Gráfico de comparação dados reais x previstos", figsize=(16,9))

# Métricas

In [ ]:
import tensorflow as tf
from sklearn.metrics import r2_score

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
print("Mean Squared Error: " + str(mse(test["30"].values[n_input:], y_pred_scaled).numpy()))
rmse = tf.keras.metrics.RootMeanSquaredError()
print("Root Mean Squared Error: " + str(rmse(test["30"].values[n_input:], y_pred_scaled).numpy()))
mae = tf.keras.losses.MeanAbsoluteError()
print("Mean Absolute Error: " + str(mae(test["30"].values[n_input:], y_pred_scaled).numpy()))

r2 = r2_score(test["30"].values[n_input:], y_pred_scaled)
print("R-Squared metric: "+ str(r2))